# Advanced Data Analysis

In [ ]:
import pandas as pd
# cal_df = pd.read_parquet('data/processed/03_calendar.parquet')
lis_df = pd.read_parquet('data/processed/03_listings.parquet') 
rev_df = pd.read_parquet('data/processed/03_reviews.parquet')
print("Listings shape:", lis_df.shape)
# print("Calendar shape:", cal_df.shape)
print("Reviews shape:", rev_df.shape)

Various types of analyses to understand the data better and to prepare for the database design.

___

## Drop dublicate superhost columns.

In [3]:
lis_df['superhost'] = lis_df['host_is_superhost']

In [7]:
lis_df.drop(['is_superhost', 'host_is_superhost'], axis=1, inplace=True)

## Drop review stats

In [9]:
lis_df.drop(['number_of_reviews_l30d','number_of_reviews_ltm'], axis=1, inplace=True)

## Drop location lat+lon after distance to center is calculated.

In [10]:
lis_df.drop(['latitude', 'longitude'], axis=1, inplace=True)

## Drop first and last review date after host experience is calculated.

In [ ]:
lis_df.drop(['first_review', 'last_review'], axis=1, inplace=True)

In [ ]:
lis_df[['host_experience_years', 'host_since']]

___

In [ ]:
price_cols = [col for col in lis_df.columns if col.startswith('yearly')]
print("Price-related columns:", price_cols)

In [ ]:
lis_df['days_since_last_review']


In [ ]:
review_cols = ['total_reviews', 'number_of_reviews', 'reviews_per_month', 'number_of_reviews_l30d', 'yearly_review', 'number_of_reviews_ltm']
print("Review-related columns from listings:")
print(lis_df[review_cols].head())


In [ ]:
lis_df[['host_listings_count', 'host_total_listings_count', 'calculated_host_listings_count', 'host_listings_ratio']][:2]